In [ ]:
# Imports all required libraries for data processing, model building, and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
import os
from sklearn.metrics import confusion_matrix, accuracy_score

# Sets up a learning rate scheduler that reduces the learning rate by 5% every epoch
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose = 0)

# Walks through the directory '/kaggle/input' and its subdirectories, and joins the directory name and file name for each file found
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [ ]:
# Loads image file paths and labels, and creates a DataFrame for the dataset
MildDemented_dir = r'/kaggle/input/alzheimers-mri-images/Processed Dataset/training/Mild Demented'
ModerateDemented_dir = r'/kaggle/input/alzheimers-mri-images/Processed Dataset/training/Moderate Demented'
NonDemented_dir = r'/kaggle/input/alzheimers-mri-images/Processed Dataset/training/Non Demented'
VeryMildDemented_dir = r'/kaggle/input/alzheimers-mri-images/Processed Dataset/training/Very Mild Demented'

filepaths = []
labels = []
dict_list = [MildDemented_dir, ModerateDemented_dir, NonDemented_dir, VeryMildDemented_dir]
class_labels = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very MildDemented']

# Iterates over the directories containing MRI images for different stages of Alzheimer's disease
for i, directory in enumerate(dict_list):
    file_list = os.listdir(directory)
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        filepaths.append(file_path)
        labels.append(class_labels[i])

# Creates pandas Series for file paths and labels
Fseries = pd.Series(filepaths, name = "filepaths")
Lseries = pd.Series(labels, name = "labels")
Alzheimer_data = pd.concat([Fseries, Lseries], axis = 1)
Alzheimer_df = pd.DataFrame(Alzheimer_data)

print(Alzheimer_df.head())
print(Alzheimer_df["labels"].value_counts())

# Shows the shape of the DataFrame containing the dataset
Alzheimer_df.shape

In [ ]:
# Splits the dataset into training, validation, and test sets
# The dataset is divided into three subsets: training, validation, and test
# Initially, the data is split into training (70%) and test (30%) sets
# Then, the training set is further divided into training (80%) and validation (20%) sets
train_images, test_images = train_test_split(Alzheimer_df, test_size = 0.3, random_state = 42)
train_set, val_set = train_test_split(Alzheimer_df, test_size = 0.2, random_state = 42)

# Prints the shapes of the training, validation, and test sets
# This code prints the number of samples in the training, validation, and test sets after the split
print(train_set.shape)
print(test_images.shape)
print(val_set.shape)
print(train_images.shape)

In [ ]:
# Creates ImageDataGenerators for training, validation, and test sets
# Initializes three data generators for training, testing, and validation using ImageDataGenerator from Keras
# Preprocesses the images by resizing and applying MobileNetV2's specific preprocessing
# Loads data from dataframes with image file paths and labels
# Sets color mode to 'rgb', class mode to 'categorical', and batch size to 32
# Disables shuffling for all generators
image_gen = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input)

train = image_gen.flow_from_dataframe(
    dataframe = train_set, x_col = "filepaths", y_col = "labels",
    target_size = (244, 244),
    class_mode = "categorical",
    batch_size = 32,
    shuffle = False
)

test = image_gen.flow_from_dataframe(
    dataframe = test_images, x_col = "filepaths", y_col = "labels",
    target_size = (244, 244),
    class_mode = "categorical",
    batch_size = 32,
    shuffle = False
)

val = image_gen.flow_from_dataframe(
    dataframe = val_set, x_col = "filepaths", y_col = "labels",
    target_size = (244, 244),
    class_mode = "categorical",
    batch_size = 32,
    shuffle = False
)

In [ ]:
# This code prints the class names that are used in the dataset
# It retrieves the class indices from the 'train' object and lists the keys, which represent the class names
# Finally, it prints out the list of class names
classes = list(train.class_indices.keys())
print(classes)

In [ ]:
# Defines a function to display a batch of images with their class labels
def show_knee_images(image_gen):
    test_dict = test.class_indices
    classes = list(test_dict.keys())
    images, labels = next(image_gen)
    plt.figure(figsize = (20, 20))
    length = len(labels)

    # Determines the number of images to display (up to 25)
    if length < 25:
        r = length
    else:
        r = 25
    # Iterates over the images in the batch
    for i in range(r):
        plt.subplot(5, 5, i + 1)
        image = (images[i] + 1) / 2
        plt.imshow(image)
        index = np.argmax(labels[i])
        class_name = classes[index]
        plt.title(class_name, color = "green", fontsize = 16)
        plt.axis('off')
    plt.show()

# Displays a batch of training images with their class labels
show_knee_images(train)

In [ ]:
# Defines and compiles the CNN model architecture for multi-class classification
model = keras.models.Sequential([
    keras.layers.Conv2D(filters = 128, kernel_size = (8, 8), strides = (3, 3), activation = 'relu', input_shape = (224, 224, 3)),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters = 256, kernel_size = (5, 5), strides = (1, 1), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size = (3, 3)),

    keras.layers.Conv2D(filters = 256, kernel_size = (3, 3), strides = (1, 1), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),
    k
    eras.layers.Conv2D(filters = 256, kernel_size = (1, 1), strides = (1, 1), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters = 256, kernel_size = (1, 1), strides = (1, 1), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size = (2, 2)),

    keras.layers.Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),

    keras.layers.MaxPool2D(pool_size = (2, 2)),

    keras.layers.Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = "same"),
    keras.layers.BatchNormalization(),

    keras.layers.MaxPool2D(pool_size = (2, 2)),
    keras.layers.Flatten(),

    keras.layers.Dense(1024, activation = 'relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(1024, activation = 'relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(4, activation = 'softmax')
])

In [ ]:
# Summarizes the model architecture and compiles the model
model.summary()

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.optimizers.SGD(learning_rate = 0.001),
    metrics = ['accuracy']
)

# Trains the CNN model using the training and validation sets
# Trains the CNN model using the provided training and validation datasets for 22 epochs
# Specifies that validation should be performed after each epoch
history = model.fit(train, epochs = 22, validation_data = val, validation_freq = 1)

# Evaluates the trained model on the test set
model.evaluate(test, verbose = 1)

# Saves the trained model to a file
model.save("alzheimer_model_cnn.h5")

In [ ]:
# Generates predictions for the test set and decodes the predicted class labels
pred = model.predict(test)
pred = np.argmax(pred, axis = 1)  # Picks class with highest probability

labels = (train.class_indices)
labels = dict((v, k) for k, v in labels.items())

pred2 = [labels[k] for k in pred]

In [ ]:
# Visualizes the evolution of model accuracy and loss during training
# Displays both metrics side by side for easy comparison between training and validation sets
fig, axs = plt.subplots(1, 2, figsize = (14, 5))

axs[0].plot(history.history['accuracy'])
axs[0].plot(history.history['val_accuracy'])
axs[0].set_title('Model Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].legend(['Train', 'Val'], loc = 'upper left')

axs[1].plot(history.history['loss'])
axs[1].plot(history.history['val_loss'])
axs[1].set_title('Model Loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].legend(['Train', 'Val'], loc = 'upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Visualizes the loss curve for both training and validation sets
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

# Prints the classification report and accuracy score for the model predictions
y_test = test_images.labels
print(classification_report(y_test, pred2))
print("Accuracy of the Model:", "{:.1f}%".format(accuracy_score(y_test, pred2) * 100))

In [ ]:
# Visualizes the confusion matrix as a heatmap with class labels
# Defines the class labels
class_labels = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very MildDemented']

# Calculates the confusion matrix
cm = confusion_matrix(y_test, pred2)

# Creates a figure and plots the confusion matrix as a heatmap
plt.figure(figsize = (10, 5))
sns.heatmap(cm, annot = True, fmt = 'g', vmin = 0, cmap = 'Blues', cbar = False)

# Sets tick labels and axis labels
plt.xticks(ticks = [0.5, 1.5, 2.5, 3.5], labels = class_labels, rotation = 30, ha = 'right', fontsize = 10)
plt.yticks(ticks = [0.5, 1.5, 2.5, 3.5], labels = class_labels, rotation = 0, fontsize = 10)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()